In [ ]:
import pandas as pd
import numpy as np
import pickle
from pipeline import Pipeline
from collections import Counter
import os

# Get model performance and genuineness

In [ ]:
pipeline_an = Pipeline(
                        trf_direction=1, 
                        trf_min_lag=0, 
                        trf_max_lag=3,
                        regularization=1,
                        modality='va',
                        audio_type='auditory_nerve'
                    )

In [ ]:
df_an = pipeline_an.make_main_df()
df_responses = pipeline_an.make_response_df()

# Make separate DataFrames for True and Fake trials
df1_an = df_an[df_an['Condition'] == 'TRUE']
df1_an = df1_an.reset_index(drop=True)
df2_an = df_an[df_an['Condition'] != 'TRUE']
df2_an = df2_an.reset_index(drop=True)

In [ ]:
trfs = pipeline_an.train_model(df1_an)

In [ ]:
with open('trfs.pkl', 'wb') as file:
    pickle.dump(trfs, file)

In [ ]:
# Predict responses to True trials
true_data = pipeline_an.predict_response(df1_an, trfs)
df_trueCorrs = pipeline_an.make_trf_df(true_data)

# Predict responses to Fake trials
fake_data = pipeline_an.predict_response(df2_an, trfs)
df_fakeCorrs = pipeline_an.make_trf_df(fake_data)

In [ ]:
df_correlations = pd.concat([df_trueCorrs, df_fakeCorrs])
df_correlations

In [ ]:
trial = []
trial_cond = []
listener_au = []
duration = []
dyad = []
times_presented = []
model_performance = []
subject_accuracy = []
genuineness = []
confidence = []

for idx, row in df_correlations.iterrows():
    if any(df_responses['VideoPath'] == row['trial']):
        ab = df_responses.loc[df_responses['VideoPath'] == row['trial']]

        subject_accuracy.append(Counter(ab['Accuracy'])[True]/len(ab))
        genuineness.append(Counter(ab['Resp'])['g']/len(ab))
        confidence.append(sum(ab['LikertResp'])/len(ab))
        trial.append(row['trial'])
        duration.append(row['duration'])
        trial_cond.append(row['condition'])
        listener_au.append(row['listener_au'])
        dyad.append(row['displayed_dyad'])
        times_presented.append(len(df_responses.loc[df_responses['VideoPath'] == row['trial']]))
        model_performance.append(row['r'])

df_regression = pd.DataFrame({
                                'trial': trial,
                                'condition': trial_cond,
                                'listener_au': listener_au,
                                'duration': duration,
                                'dyad': dyad,
                                'times_presented': times_presented,
                                'model_performance': model_performance,
                                'subject_accuracy': subject_accuracy,
                                'confidence': confidence,
                                'genuineness': genuineness
                            })
df_regression

In [ ]:
df = df_regression

In [ ]:
def get_percentile(df, au, col, percent):
    return df[df['listener_au']==au][col].quantile(percent)

In [ ]:
def get_description(df, au, col):
    return df[df['listener_au']==au][col].describe()

# Hits

In [ ]:
hit_eyes = df[
    (df['condition']=='true') & 
    (df['listener_au']=='AU43') & 
    (df['genuineness'] > get_percentile(df, 'AU43', 'genuineness', .70)) & 
    (df['model_performance'] > get_percentile(df, 'AU43', 'model_performance', .70))
].reset_index(drop=True)

## AU17

In [ ]:
hit_17 = df[
    (df['condition']=='true') & 
    (df['listener_au']=='AU17') & 
    (df['genuineness'] > get_percentile(df, 'AU17', 'genuineness', .70)) & 
    (df['model_performance'] > get_percentile(df, 'AU17', 'model_performance', .70))
].reset_index(drop=True)

In [ ]:
hit_17 = pd.merge(hit_17, hit_eyes, 'inner', on='trial')
# hit_17

## AU25

In [ ]:
hit_25 = df[
    (df['condition']=='true') & 
    (df['listener_au']=='AU25') & 
    (df['genuineness'] > get_percentile(df, 'AU25', 'genuineness', .70)) & 
    (df['model_performance'] > get_percentile(df, 'AU25', 'model_performance', .70))
].reset_index(drop=True)

In [ ]:
hit_25 = pd.merge(hit_25, hit_eyes, 'inner', on='trial')
# hit_25

## Combined

In [ ]:
df_hit = pd.concat([hit_17, hit_25]).drop_duplicates(subset='trial').reset_index(drop=True)
df_hit['sdt'] = np.repeat(['hit'], len(df_hit))
df_hit

# Misses

In [ ]:
miss_eyes = df[
    (df['condition']=='true') & 
    (df['listener_au']=='AU43') & 
    (df['genuineness'] < get_percentile(df, 'AU43', 'genuineness', .41)) & 
    (df['model_performance'] < get_percentile(df, 'AU43', 'model_performance', .60))
].reset_index(drop=True)

## AU17

In [ ]:
miss_17 = df[
    (df['condition']=='true') & 
    (df['listener_au']=='AU17') & 
    (df['genuineness'] < get_percentile(df, 'AU17', 'genuineness', .41)) & 
    (df['model_performance'] > get_percentile(df, 'AU17', 'model_performance', .60))
].reset_index(drop=True)

In [ ]:
miss_17 = pd.merge(miss_17, miss_eyes, 'inner', on='trial')
# miss_17

## AU25

In [ ]:
miss_25 = df[
    (df['condition']=='true') & 
    (df['listener_au']=='AU25') & 
    (df['genuineness'] < get_percentile(df, 'AU25', 'genuineness', .41)) & 
    (df['model_performance'] > get_percentile(df, 'AU25', 'model_performance', .60))
].reset_index(drop=True)

In [ ]:
miss_25 = pd.merge(miss_25, miss_eyes, 'inner', on='trial')
# miss_25

## Combined

In [ ]:
df_miss = pd.concat([miss_17, miss_25]).drop_duplicates(subset='trial').reset_index(drop=True)
df_miss['sdt'] = np.repeat(['miss'], len(df_miss))
df_miss

# Correct Rejections

In [ ]:
cr_eyes = df[
    (df['condition']=='fake') & 
    (df['listener_au']=='AU43') & 
    (df['genuineness'] < get_percentile(df, 'AU43', 'genuineness', .35)) & 
    (df['model_performance'] > get_percentile(df, 'AU43', 'model_performance', .70))
].reset_index(drop=True)

## AU17

In [ ]:
cr_17 = df[
    (df['condition']=='fake') & 
    (df['listener_au']=='AU17') & 
    (df['genuineness'] < get_percentile(df, 'AU17', 'genuineness', .35)) & 
    (df['model_performance'] < get_percentile(df, 'AU17', 'model_performance', .30))
].reset_index(drop=True)

In [ ]:
cr_17 = pd.merge(cr_17, cr_eyes, 'inner', on='trial')
# cr_17

## AU25

In [ ]:
cr_25 = df[
    (df['condition']=='fake') & 
    (df['listener_au']=='AU25') & 
    (df['genuineness'] < get_percentile(df, 'AU25', 'genuineness', .35)) & 
    (df['model_performance'] < get_percentile(df, 'AU25', 'model_performance', .30))
].reset_index(drop=True)

In [ ]:
cr_25 = pd.merge(cr_25, cr_eyes, 'inner', on='trial')
# cr_25

## Combined

In [ ]:
df_cr = pd.concat([cr_17, cr_25]).drop_duplicates(subset='trial').reset_index(drop=True)
df_cr['sdt'] = np.repeat(['cr'], len(df_cr))
df_cr

# False Alarms

In [ ]:
fa_eyes = df[
    (df['condition']=='fake') & 
    (df['listener_au']=='AU43') & 
    (df['genuineness'] > get_percentile(df, 'AU43', 'genuineness', .50)) & 
    (df['model_performance'] < get_percentile(df, 'AU43', 'model_performance', .50))
].reset_index(drop=True)

## AU17

In [ ]:
fa_17 = df[
    (df['condition']=='fake') & 
    (df['listener_au']=='AU17') & 
    (df['genuineness'] > get_percentile(df, 'AU17', 'genuineness', .50)) & 
    (df['model_performance'] > get_percentile(df, 'AU17', 'model_performance', .50))
].reset_index(drop=True)

In [ ]:
fa_17 = pd.merge(fa_17, fa_eyes, 'inner', on='trial')
# fa_17

## AU25

In [ ]:
fa_25 = df[
    (df['condition']=='fake') & 
    (df['listener_au']=='AU25') & 
    (df['genuineness'] > get_percentile(df, 'AU25', 'genuineness', .50)) & 
    (df['model_performance'] > get_percentile(df, 'AU25', 'model_performance', .50))
].reset_index(drop=True)

In [ ]:
fa_25 = pd.merge(fa_25, fa_eyes, 'inner', on='trial')
# fa_25

## Combined

In [ ]:
df_fa = pd.concat([fa_17, fa_25]).drop_duplicates(subset='trial').reset_index(drop=True)
df_fa['sdt'] = np.repeat(['fa'], len(df_fa))
df_fa

# Final trial list

In [ ]:
df_final = pd.concat([df_hit, df_miss, df_cr, df_fa]).reset_index(drop=True)

In [ ]:
selector_dict = {'trial': 'video_path', 'condition_x': 'condition', 'duration_x': 'duration', 'dyad_x': 'dyad', 'model_performance_x': 'mouth_fit', 'model_performance_y': 'eye_fit', 'genuineness_x': 'genuineness', 'sdt': 'category'}

df_final = df_final.rename(columns=selector_dict)[[*selector_dict.values()]]

In [ ]:
block = np.tile(np.asarray(['original', 'mouth', 'eyes', 'nods']).reshape(1, -1), (len(df_final), 1))
df_final['block'] = list(block)
df_final = df_final.explode(column='block')

In [ ]:
df_final['video_path'] = df_final.apply(lambda x: x['video_path'].replace(os.sep, '/'), axis=1)

df_final['audio_path'] = df_final.apply(lambda x: f"./stimuli/audio/{x['dyad']}/{x['video_path'].split('/')[5][:-7]}_audio.wav", axis=1)
df_final['video_path'] = df_final.apply(lambda x: f"./stimuli/{x['block']}/{'_'.join(x['video_path'].split('/')[4:])[:-7]}.mov", axis=1)

In [ ]:
df_final

In [ ]:
exclude = [
            './stimuli/audio/11/8_audio.wav', './stimuli/audio/24/4_audio.wav',     # misses
            './stimuli/audio/49/2_5_audio.wav', './stimuli/audio/11/1_2_audio.wav', './stimuli/audio/18/1_3_audio.wav'      # correct rejections
        ]

df_practice = df_final[df_final['audio_path'].isin(exclude)].reset_index(drop=True)
df_practice

df_practice.to_csv('./practice_trials.csv', index=False)

In [ ]:
df_final = df_final[~df_final['audio_path'].isin(exclude)].reset_index(drop=True)
df_final

In [ ]:
df_final.to_csv('./trial_list.csv', index=False)